# 2유형_분류모델

## 예
1. 고객성별예측(남자일 확률)
2. 타이타닉 생존 여부(생존, 사망)

## 사용 가능 모델
1. XGBClassifier
2. RandomForestClassifier
3. DecisionTreeClassifier
4. KNeighborsClassifier 사용

# 2유형_예측모델

## 예
1. 자전거 수요 예측

## 사용 가능 모델
1. XGBRegressor
2. LinearRegression 사용
3. RandomForestRegressor 사용

#### print()하고 나서 더이상 쓸모없는 print문은 주석처리

### 1. 데이터 탐색
- 4개 단계 모두 진행

#### 1) 데이터 불러오기(x_train, x_test, y_train)

In [ ]:
import pandas as pd
pd.options.display.max_columns = None     # 전체 칼럼 출력 옵션
import warnings
warnings.filterwarnings(action = 'ignore')     # warnings 무시 옵션 사용 가능?

x_train = pd.read_csv('x_train.csv', encoding = 'cp949')
x_test = pd.read_csv('x_test.csv', encoding = 'cp949')
y_train = pd.read_csv('y_train.csv', encoding = 'cp949')

print(x_train.head())
print(x_test.head())
print(y_train.head())

#### 2) info확인(x_train, x_test, y_train)

- 결측치 존재 확인
- 칼럼별 타입 확인(object). 특히, datetime같은 날짜 관련 칼럼 확인
- 범주형 변수 존재 확인(봄여름가을겨울, 공휴일또는공휴일아님 등)

In [ ]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

#### 3) describe확인(x_train)
- 칼럼별 mean 확인해서 차이가 큰지 안큰지 확인
    - 칼럼별 mean이 0 ~ 100 사이면 스케일링 필요 없음
    - 하지만 그 이상 차이가 나면 스케일링 필요함(각 칼럼의 범위를 동일한 범위로 변환 필요)

In [ ]:
print(x_train.describe())

#### 4) 독립변수와 종속변수의 관계 확인(범주형 변수만)
- 종속변수에 중요한 독립변수의 영향도와 추세 파악
- 이를 위해 x_train, y_train 통합해야 함


- 각 칼럼별로 칼럼에 따른 종속변수의 평균을 계산해 봤을 때, 등급이 높을수록 생존율이 높다든가, 성별에 따라 생존율의 차이가 생긴다든가, 선착장에 따라 생존율의 차이가 생기면 유용한 칼럼이므로 분석 대상에 포함
- 그렇지 않다면(칼럼의 데이터 종류별 생존율이 비슷하다면) 분석 대상에서 제외해도 됨 

In [ ]:
# x_train, y_train 통합
data = pd.concat([x_train, y_train], axis = 1)
print(data.head())

In [ ]:
# 칼럼 중 종속변수칼럼과 범주형 칼럼들 확인 - 각 칼럼별로 확인
print(data.groupby(['독립변수 칼럼명'])['종속변수 칼럼명'].mean())     # 생존여부
# 또는
print(data.groupby(['독립변수 칼럼명'])['종속변수 칼럼명'].sum())     # 수요예측

### 2. 전처리
- 전처리의 7개 단계를 모두 해야 하는 것은 아님. 필요한 부분만 진행

#### cf) datetime 같은 날짜 칼럼 존재 할 경우
- object 타입이면 datetime 타입으로 변경 후
- year, month, day, hour, dayofweek(요일) 칼럼 생성 후 어떤 데이터가 있는지 확인
- 추가한 칼럼 대상으로 다시 독립변수와 종속변수의 관계 확인 후 필요없는 칼럼 제거

In [ ]:
# datetime 칼럼을 datetime으로 변경
x_train['datetime'] = pd.to_datetime(x_train['datetime'])

x_train['year'] = x_train['datetime'].dt.year
x_train['month'] = x_train['datetime'].dt.month
x_train['day'] = x_train['datetime'].dt.day
x_train['hour'] = x_train['datetime'].dt.hour
x_train['dayofweek'] = x_train['datetime'].dt.dayofweek
print(x_train.head())

print(x_train['year'].unique())
print(x_train['month'].unique())
print(x_train['day'].unique())
print(x_train['hour'].unique())
print(x_train['dayofweek'].unique())

# 새로 만든 x_train을 다시 y_train과 합쳐서 다시 독립변수와 종속변수의 관계 확인(추가한 칼럼만)
data2 = pd.concat([x_train, y_train], axis = 1)
print(data2.head())
print(data2.groupby(['추가한 칼럼명'])['종속변수 칼럼명'].sum())     # 또는 mean()
# 특별한 영향이 없는 칼럼은 제거
x_train = x_train.drop(['필요없는 칼럼명'], axis = 1)
print(x_train.head())

# 최종적으로 새로 생긴 칼럼은 테스트 데이터에도 동일하게 적용
x_test['datetime'] = pd.to_datetime(x_test['datetime'])
x_test['year'] = x_test['datetime'].dt.year
x_test['hour'] = x_test['datetime'].dt.hour

#### 1) 불필요한 칼럼 삭제(x_train, x_test, y_train)

In [ ]:
# id칼럼 또는 datetime칼럼 같은 경우는 예측에 아무런 영향이 없으므로 삭제
# 단, x_test데이터의 id칼럼 또는 datetime칼럼은 최종 제출할 때 필요하므로 따로 저장

x_test_id칼럼명 또는 datetime칼럼명 = x_test['id칼럼명 또는 datetime칼럼']

x_train = x_train.drop(columns = ['id칼럼명 또는 datetime칼럼 '])
x_test = x_test.drop(columns = ['id칼럼명 또는 datetime칼럼 '])
y_train = y_train.drop(columns = ['id칼럼명 또는 datetime칼럼 '])

print(x_train.head())
print(y_train.head())
print(x_test.head())

In [ ]:
# object 타입 칼럼 중 칼럼의 데이터 종류가 너무 많은 칼럼은 삭제 처리

print(x_train['object 칼럼명'].nunique())     # 이 결과값이 너무 많이 나오면 삭제
x_train = x_train.drop(['칼럼명1', '칼럼명2'], axis = 1)
x_test = x_test.drop(['칼럼명1', '칼럼명2'], axis = 1)

print(x_train.head())
print(y_train.head())
print(x_test.head())

#### 2) 결측치 처리(x_train, x_test)

- 예를들어, 환불금액 같은 경우 결측치가 매우 많으면(65%) 환불 이력이 없는 데이터가 결측치 일 수 있으므로 0으로 대체

In [ ]:
# 결측치 확인
print(x_train.isna().sum())

# 결측치 비율 확인
print(x_train.isna().sum() / len(x_train))

In [ ]:
# 결측치 처리 방법

# 1. 결측치 존재 칼럼의 결측치 0으로 대체
x_train['결측치 존재 칼럼명'] = x_train['결측치 존재 칼럼명'].fillna(0)
x_test['결측치 존재 칼럼명'] = x_test['결측치 존재 칼럼명'].fillna(0)

# 2. 결측치 존재 칼럼과 종속변수 칼럼의 상관성 확인 후 삭제
print(data[['결측치 존재 칼럼명', '종속변수 칼럼명']].corr())     # 절댓값이 0에 가까우면 0.07정도 삭제
x_train = x_train.drop(['삭제할 칼럼명'], axis = 1)
x_test = x_test.drop(['삭제할 칼럼명'], axis = 1)

# 3. 결측치 비율이 매우 높고(0.77정도) 해당 칼럼의 데이터 종류도 많으면 삭제
x_train = x_train.drop(['삭제할 칼럼명'], axis = 1)
x_test = x_test.drop(['삭제할 칼럼명'], axis = 1)

# 4. 결측치가 매우 적고(2개정도) 범주형 칼럼이면 가장 많은 비율을 차지하는 값으로 대체
x_train['결측치 존재 칼럼명'].value_counts()     # 가장 많은 비율을 차지하는 값 찾아서 해당 값으로 대체
x_train['결측치 존재 칼럼명'] = x_train['결측치 존재 칼럼명'].fillna('가장많은데이터값')
x_test['결측치 존재 칼럼명'] = x_test['결측치 존재 칼럼명'].fillna('가장많은데이터값')

In [ ]:
# 결측치 처리 잘 됐나 확인
print(x_train.isna().sum())
print(x_test.isna().sum())

#### 3) 범주형 변수 인코딩(x_train, x_test)

1. 범주형 변수의 데이터의 종류를 확인 => x_train['칼럼명'].nunique()
2. 범주형 데이터의 데이터 종류가 매우 작으면(2개 예. 성별) replace로 변환
3. 데이터 종류의 갯수가 적으면 (10개 이하) 원핫 인코딩
4. 그렇지 않으면 라벨 인코딩

In [ ]:
# 범주형 변수 확인(각 칼럼별 데이터 종류 갯수 확인)

print(x_train['범주형 칼럼명'].nunique())

In [ ]:
# 1. 범주형 데이터 칼럼의 데이터 종류가 2개 이내면 예를 들어, male = 0, female = 1로 변환

x_train['범주형 칼럼명'] = x_train['범주형 칼럼명'].replace('male', 0).replace('female', 1)
x_test['범주형 칼럼명'] = x_test['범주형 칼럼명'].replace('male', 0).replace('female', 1)

In [ ]:
# 2. 범주형 변수의 데이터 종류가 적으면 (10개 이내) 원핫인코딩. 예: 선착장 칼럼(Q, S, C 존재)

dummy = pd.get_dummies(x_train['범주형 칼럼명'], 
                           drop_first = True).rename(columns = {'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train, dummy], axis = 1)
x_train = x_train.drop(columns = ['범주형 칼럼명'])

dummy = pd.get_dummies(x_test['범주형 칼럼명'], 
                           drop_first = True).rename(columns = {'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test, dummy], axis = 1)
x_test = x_test.drop(columns = ['범주형 칼럼명'])
print(x_train.head())
print(x_test.head())

In [ ]:
# 3. 범주형 변수의 칼럼의 라벨인코딩(train, test 데이터 모두) - 칼럼이 여러개면 여러번 반복

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

x_train['범주형 칼럼명'] = encoder.fit_transform(x_train['범주형 칼럼명'])
x_test['범주형 칼럼명'] = encoder.fit_transform(x_test['범주형 칼럼명'])

print(x_train.head())

# 라벨인코딩 변환 결과 확인
print(encoder.classes_)     # 이 구문 결과의 순서대로 0, 1, 2 등의 순서로 라벨인코딩 됨

#### 4) 파생변수 만들기(x_train, x_test)

- 예를 들어,
- 금액 > 0이면 1, 금액 = 0이면 0으로 파생변수 생성하든가
- 배우자수 칼럼 + 부모자식수 칼럼 = 가족수라는 파생변수 생성하든가

In [ ]:
# 환불금액 칼럼에 대해서 환불금액>0이면 1, 환불금액 = 0이면 0으로 파생변수 생성
condition = x_train['환불금액'] > 0

x_train.loc[condition, '환불금액_new'] = 1     
x_train.loc[~condition, '환불금액_new'] = 0     
x_train = x_train.drop(columns = ['환불금액'])

# 테스트 데이터 세트에도 동일하게 적용
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0
x_test = x_test.drop(columns = ['환불금액'])

print(x_train.head())
print(x_test.head())

#### 5) 표준화(데이터 스케일링)(x_train, x_test)

In [ ]:
# 1. StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 2. MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler

In [ ]:
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

print(x_train.describe())
# 표준화 진행 후 각 변수의 평균값이 0에 가깝고, 표준편차는 1에 근접함을 확인할 수 있음

#### 6) 상관관계 확인(x_train, x_test)
- 예를 들어, 독립변수 중 '금액'이 포함된 칼럼 간의 상관성 확인
- 상관성 확인하여 상관성이 높은 변수(절댓값이 0.6이상)이면 둘 중 한 칼럼은 삭제

In [ ]:
print(x_train[['칼럼명', '다른칼럼명', '또있으면또다른칼럼명']].corr())

# 상관계수의 절댓값이 0.6이상인 칼럼이 있으면 둘 중 한 칼럼 삭제
# train, test 모두에서 칼럼 삭제
x_train = x_train.drop(columns = ['삭제할 칼럼명'])
x_test = x_test.drop(columns = ['삭제할 칼럼명'])

### 3. 학습하고 평가하기

#### 1) train 데이터를 학습용과 테스트로 나누기(모델 평가를 위해)

In [ ]:
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2022)

print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

#### 2) 데이터 학습

In [ ]:
# 분류모델(성별 예측, 생존 여부 예측)
# 1. XGBClassifier 사용
from xgboost import XGBClassifier
model = XGBClassifier(eval_metric = 'error', random_state = 2022, max_depth = 5, n_estimators = 100)

# 2. RandomForestClassifier 사용
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 2022, max_depth = 5, n_estimators = 200, max_leaf_nodes = 10)

# 3. DecisionTreeClassifier 사용
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state = 2022, max_depth = 10, criterion = 'entropy')

# 4. KNeighborsClassifier 사용
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')


# 예측모델(수요 예측)
# 1. XGBRegressor 사용
from xgboost import XGBRegressor
model = XGBRegressor(random_state = 2022, max_depth = 5, n_estimators = 200)

# 2. LinearRegression 사용
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# 3. RandomForestRegressor 사용
from sklearn.ensemble import RandonForestRegressor
model = RandomForestRegressor(random_state = 2022, max_depth = 5, n_estimators = 200)

In [ ]:
model.fit(X_TRAIN, Y_TRAIN)
# 혹시 여기서 DataConversion warning가 난다면 Y_TRAIN.values.ravel()로 변경

#### 3) 모델 평가
1. roc_auc_score
2. r2_score
3. mean_squared_error

In [ ]:
# roc_auc_score는 1에 가까울수록 모델 성능 우수
# 모델 성능 평가를 위한 코드
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

In [ ]:
# R^2 score
# 모델 성능 평가를 위한 코드
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST)).rename(columns = {0:'count'}) # 칼럼명 변경 필요시 변경
Y_TEST_PREDICTED[Y_TEST_PREDICTED['count'] < 0 ] = 0     # 수요예측이 0미만이면 안될 경우 작성

from sklearn.metrics import r2_score
print(r2_score(Y_TEST, Y_TEST_PREDICTED))

In [ ]:
# mean_squared_error
# 모델 성능 평가를 위한 코드
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

from sklearn.metrics import mean_squared_error
print(mean_squared_error(Y_TEST, Y_TEST_PREDICTED))

# RMSE는 MSE에 제곱근을 씌운 형태이므로
print(mean_squared_error(Y_TEST, Y_TEST_PREDICTED, squared = False))

#### 4) 결과 예측

In [ ]:
# 1. 만약 성별 여부(0인지 1인지)가 아니라 확률(0일 확률)을 구해야 하면 predict_proba()함수 사용
y_test_proba = model.predict_proba(x_test)
print(pd.DataFrame(y_test_proba).head())

# 최종적으로 구해야 하는 확률은 남성(1)이므로 남성 성별로 분류된 확률 확인
print(pd.DataFrame(y_test_proba)[1])

# 위 결과를 result 변수에 저장
result = pd.DataFrame(result)[1]
print(result)


# 2. 만약 생존여부처럼 (0, 1)을 분류해야 하면 predict()함수 사용
y_test_predicted = model.predict(x_test)
y_test_predicted = pd.DataFrame(y_test_predicted)
print(y_test_predicted)

# 위 결과를 result 변수에 저장
result = y_test_predicted
print(result)


# 3. 만약 수요예측처럼 예측을 해야 하는 경우면 predict()함수 사용
y_test_predicted = model.predict(x_test)    
y_test_predicted = pd.DataFrame(y_test_predicted)     
y_test_predicted = y_test_predicted.rename(columns = {0:'count'})
# count예측값이 음수로 되는 경우가 있으므로 음수는 모두 0으로 변경
y_test_predicted[y_test_predicted['count'] < 0] = 0

# 위 결과를 result 변수에 저장
result = y_test_predicted
print(result)

### 4. 결과 제출
- 문제에서 제시한 최종결과와 동일하게 출력해야 함

In [ ]:
# 최종 데이터는 id칼럼명 또는 datetime칼럼명과 y_test 예측값을 함께 출력해야 함
# concat() 함수를 이용해 x_test_id칼럼명 또는 datetime칼럼명 변수와 result 변수를 통합
pd.concat([x_test_id칼럼명 또는 datetime칼럼명, result], axis = 1)

# 1 칼럼명을 gender로 변경해야 함(필요시)
pd.concat([x_test_id칼럼명 또는 datetime칼럼명, result], axis = 1).rename(columns = {1:'gender'})

# 최종파일로 제출하기
# 행 번호인 index는 저장하지 않기 위해 index = False 옵션 필수
final = pd.concat([x_test_id칼럼명 또는 datetime칼럼명, result], axis = 1).rename(columns = {1:'gender'})
final.to_csv('수험번호.csv', index = False)

# 제출된 파일 정상 여부 확인
answer = pd.read_csv('수험번호.csv')
print(answer)